# Modeling Strange Plasmas with Positrons

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import uuid
from collections import defaultdict

The code up to this point, only keeps track of teh current location of the drunk. What if we were interested in the path a drunk takes? Then we need to have something like a list that we continually update. An immediate question we need to address is, "Where should we keep the list? Should it be part of the `Field` class or one of the `Drunk` classes. 

In [ ]:
import numbers
import math
import random
import seaborn as sns


#### Create a `DiffusionField` class to approximate diffusion behavior

In [ ]:
import numbers
import math
import random
class Location(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
            
    @property
    def x(self):
        return self.__x
    @x.setter
    def x(self, value):
        if isinstance(value, numbers.Real):
            self.__x = value
        else:
            raise ValueError("location values must be real numbers")
    @property
    def y(self):
        return self.__y
    @y.setter
    def y(self, value):
        if isinstance(value, numbers.Real):
            self.__y = value
        else:
            raise ValueError("location values must be real numbers")
            
    def distFrom(self, other):
        return math.sqrt((self.x-other.x)**2+(self.y-other.y)**2)
    
    def move(self, deltaX, deltaY):
        return Location(self.x+deltaX, self.y+deltaY)
    
    def __str__(self):
        return "<%s,%s>"%(str(self.x), str(self.y))
    def __add__(self, other):
        return  Location(self.x+other.x, self.y+other.y)
class Space(object):
    def __init__(self):
        self.__particles = {}
        
    def particles(self):
        return tuple(self.__particles.keys())
    def hasParticle(self, particle):
        return particle in self.__particles
    
    def addParticle(self, particle, loc):
        
        if not isinstance(loc, Location):
            raise TypeError("loc must be an instance of location")
        if self.hasParticle(particle):
            raise ValueError("Duplicate particle")
                            
        else:
            self.placeParticle(particle, loc)
    def removeParticle(self,particle):
        del self.__particles[particle]
    def placeParticle(self, particle, loc):
        self.__particles[particle] = loc
        
    def iterateParticle(self, particle):
        if particle not in self.__particles:
            raise ValueError("particle not in field")
        xDist, yDist = particle.iterate()
        currentLocation = self.__particles[particle]
        self.placeParticle(particle, currentLocation.move(xDist, yDist))
        
        
    def getLoc(self, particle):
        if particle not in self.__particles:
            raise ValueError("particle not in field")
        return self.__particles[particle]
                            

In [ ]:
import collections
class DiffusionField(Space):
    def __init__(self):
        super(DiffusionField, self).__init__()
        self.__occupied = defaultdict(list)

    def isOccupied(self, loc):
        return loc in self.__occupied
    
    def iterateParticle(self, particle):
        if not self.hasParticle(particle):
            raise ValueError("Particle not in field")
        rslt = particle.iterate()
        currentLocation = self.getLoc(particle)
        if len(rslt) == 3:
            p1, p2, _ = rslt
            self.removeParticle(particle)
            self.__occupied[currentLocation].remove(particle)
            self.__occupied[currentLocation].extend([p1,p2])
            self.placeParticle(p1, currentLocation)
            self.placeParticle(p2, currentLocation)
        else:
            xDist, yDist = rslt
            currentLocation = self.getLoc(particle)
            newLocation = currentLocation.move(xDist, yDist)

            if len(self.__occupied[particle]) > 0:
                neighbors = self.__occupied[newLocation]
                for n in neighbors:
                    if isinstance(particle, electron):
                        if isinstance(n, electron):
                            if particle.charge != neighbor.charge:
                                self.removeParticle(particle)
                                self.removeParticle(n)
                                p1, p2 = photon(), photon()
                                self.__particles[p1] = newLocation
                                self.__particles[p2] = newLocation
                                del neighbors[n]
                                neighbors.extend([p1,p2])
                            else:
                                particle.cool()
                                n.heat()
                        elif isinstance(n, proton):
                            if random.random() < n.electron_capture_rate:
                                a = atom(particle, n)
                                del neighbors[n]
                                self.removeParticle(particle)
                                self.removeParticle(n)
                                neighbors.append(a)
                                self.placeParticle(a, newLocation)
                            else:
                                particle.heat()
                        else:
                            particle.cool()
                                
                    elif isinstance(particle, photon):
                        if random.random() < n.capture_rate:
                            n.heat()
                            self.removeParticle(particle)
                self.placeParticle(particle, currentLocation)
            else:
                self.__occupied[currentLocation].remove(particle)
                self.__occupied[newLocation].append(particle)
                #print("new location")
                self.placeParticle(particle, newLocation)
            
    def addParticle(self, particle, loc):
        
        if loc in self.__occupied:
            raise ValueError("occupied space")
        if not isinstance(loc, Location):
            raise TypeError("loc must be an instance of location")
        if self.hasParticle(particle):
            raise ValueError("Duplicate particle")
                            
        else:
            self.__occupied[loc].append(particle)
            self.placeParticle(particle, loc)            
    
        

In [ ]:
import random
import numbers
class Particle(object):
    stepSize = 0
    stepChoices = [(0,1), (0,-1), (1, 0), (-1, 0)]
    mass = 0
    charge = 0
    capture_rate = 0
    def __init__(self, pid=None):
        if pid == None:
            self.pid = uuid.uuid1().int
        else:
            self.pid=pid
        self.stepSize = 0
    def __takeStep(self):
        step = random.choice(self.stepChoices)
        return tuple([self.stepSize* s for s in step])
    def __str__(self):
        return str(self.pid)
    def __repr__(self):
        return self.__str__()
        
    
    @property
    def pid(self):
        return self.__pid
    @pid.setter
    def pid(self,pid):
        if not isinstance(pid,int):
            raise TypeError("identifier must be an integer")
        if pid < 0:
            raise ValueError("identifer must be a non-negative integer")
        self.__pid = pid
        
    def __str__(self):
        if self != None:
            return str(self.pid)
        return "-1"
    def iterate(self):
        return self.__takeStep()
    def __takeStep(self):
        return random.choice(self.stepChoices)
    def heat(self):
        self.stepSize += 1
        
    def cool(self):
        self.stepSize -= 1
    @property   
    def kineticEnergy(self):
        return 0.5*self.mass+self.stepSize**2
        
class photon(Particle):
    mass = 0
    stepSize = 10
    
class electron(Particle):
    mass = 1
    charge = -1
    capture_rate = 0.2
    def __init__(self,*args, **kwargs):
        self.stepSize = 5
        super(electron, self).__init__(*args, **kwargs)
    def heat(self):
        self.stepSize += 2
    def cool(self):
        self.stepSize -= 2
        

class Positron(electron):
    charge = 1
    
class Proton(Particle):
    mass = 1800
    charge = 1
    decay_rate = 0.01
    electron_capture_rate = 0.2
    capture_rate = 0.01
    
    def __takeStep(self):
        stepChoices = [(0,1), (0,-1), (1, 0), (-1, 0)]
        return random.choice(stepChoices)
    def decay(self):
        return neutron(pid=uuid.uuid1().int), Positron(pid=uuid.uuid1().int), self.pid
    def iterate(self):
        if random.random() < self.decay_rate:
            return self.decay()
        else:
            return self.__takeStep()
        
class neutron(Particle):
    charge = 0
    mass = 1800
    capture_rate = 0.01
class atom(list, Particle):
    capture_rate = 0.01
    def __init__(self, *args, pid=1, **kwargs):
        self.pid = pid
        super(atom,self).__init__(*args, **kwargs)
    
    @property
    def mass(self):
        return sum([m.mass for m in self])
    @property
    def charge(self):
        return sum([s.charge for s in self])
    @property
    def kineticEnegy(self):
        return sum([s.kineticEnergy for s in self])
    
    def __str__(self):
        return ":".join([s.__str__() for s in self])
    def __repr__(self):
        return self.__str__()
    def __hash__(self):
        return hash(self.__repr__())

In [ ]:
def getLoc(f, xr, yr):
    loc = Location(random.randint(-xr, xr), random.randint(-yr, yr))
    if not f.isOccupied(loc):
        return loc
    else:
        return get_loc(f, xr, yr)
    
def populatField(f, numParticles, xRange, yRange):
    for i in range(numParticles):
        r = random.random()
        if r < 0.01:
            p = Positron()
        elif r < 0.21:
            p = electron()
        elif r < 0.41:
            p = Proton()
        elif r <0.51:
            p = neutron()
        elif r < 0.8:
            p1, p2 = Proton(), electron()
            p = atom([p1,p2])
        else:
            p = photon()
        f.addParticle(p, getLoc(f, xRange, yRange))

    return len(f.particles())

In [ ]:
def viewField(f, xrange, yrange):
    def _get_color(p):
        if isinstance(p, Positron):
            return 'red'
        elif isinstance(p, electron):
            return 'blue'
        elif isinstance(p, Proton):
            return 'orange'
        elif isinstance(p, neutron):
            return 'brown'
        elif isinstance(p, atom):
            return 'green'
        else:
            return 'black'
    def _get_size(p):
        if isinstance(p, photon):
            return 2
        elif isinstance(p, electron):
            return 2
        elif isinstance(p, atom):
            return 10
        else:
            return 5
        
    xvals, yvals, colors, sz = [], [], [], []
    for p in f.particles():
        loc = f.getLoc(p)
        xvals.append(loc.x)
        yvals.append(loc.y)
        colors.append(_get_color(p))
        sz.append(_get_size(p))

    plt.scatter(xvals, yvals, marker = 'o', color=colors, s=sz)
    plt.xlim(-xrange, xrange)
    plt.ylim(-yrange, yrange)

In [ ]:

def run_field(f, numSteps):
    for i in range(numSteps):
        for p in f.particles():
            f.iterateParticle(p)
    return None

## Run a diffusion process

In [ ]:
f = DiffusionField()
populatField(f, 50, 200, 200)
viewField(f, 800, 800)

In [ ]:
run_field(f, 3000)

viewField(f, 800, 800)

### Questions we could ask about this simulation

* Evolution of energy
* Evolution of mass
* Diffusion